# Secure credentials with Bedrock AgentCore Identity

This workshop demonstrates how to integrate Strands Agents with Amazon Bedrock AgentCore Identity to securely manage API keys and credentials for external services when building AI agents.

## Overview

In this lab, you will:
- Learn about secure credential management challenges
- Understand Bedrock AgentCore Identity capabilities
- Create API Key Credential Providers for external services
- Test secure credential retrieval in AI agents
- Explore best practices for credential management

## Prerequisites

Before starting this lab, ensure you have:
- AWS credentials configured (IAM role or environment variables)
- Required Python packages installed
- Basic understanding of Strands Agents and Bedrock AgentCore concepts
- An external API key (e.g., Exa API key) for testing

If you're not running in an environment with an IAM role assumed, set your AWS credentials as environment variables:

In [ ]:
import os

#os.environ["AWS_ACCESS_KEY_ID"]=<YOUR ACCESS KEY>
#os.environ["AWS_SECRET_ACCESS_KEY"]=<YOUR SECRET KEY>
#os.environ["AWS_SESSION_TOKEN"]=<OPTIONAL - YOUR SESSION TOKEN IF TEMP CREDENTIAL>
#os.environ["AWS_REGION"]=<AWS REGION WITH BEDROCK AGENTCORE AVAILABLE>

Install required packages for Strands Agents and Bedrock AgentCore Python SDK:

In [ ]:
%pip install -q strands-agents strands-agents-tools bedrock-agentcore

## Retrieve Exa API key to connect Remote Exa MCP

In this lab, we will interact [Remote Exa MCP](https://docs.exa.ai/reference/exa-mcp) to perform real-time web searches through the Exa Search API, which requires Exa API key to connect.

Exa MCP server URL: ```https://mcp.exa.ai/mcp?exaApiKey=your-exa-api-key```

To get Exa API key, go to [Exa login page](https://dashboard.exa.ai/login) to register with your email. 

Then go to [API key section](https://dashboard.exa.ai/api-keys) in Exa dashboard to create an API key. Copy the API key to `EXA_API_KEY` in below code...

## Understanding Credential Management Challenges

### Demonstrating Insecure API Key Usage

Let's first demonstrate what happens when we try to use an external service (Exa search) without proper credential management. This will show the security risks and authentication failures that occur with hardcoded or invalid API keys.

In [ ]:
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient
from mcp.client.streamable_http import streamablehttp_client

# !-------- UPDATE THE EXA API KEY HERE  --------!
EXA_API_KEY = <YOUR EXA API KEY> 

# Connect to the weather MCP server
print("\nConnecting to MCP Server...")
exa_server = MCPClient(lambda: streamablehttp_client(f"https://mcp.exa.ai/mcp?exaApiKey={EXA_API_KEY}"))

with exa_server:
    # Combine all tools - they all work the same way!
    mcp_tools = (
        exa_server.list_tools_sync()
    )

    print(f"Available tools: {[tool.tool_name for tool in mcp_tools]}")
    
    # Create agent with Exa MCP tools
    agent = Agent(model=BedrockModel(model_id="us.amazon.nova-pro-v1:0"),
                  system_prompt="You are a helpful assistant that provides concise responses.",
                  tools=mcp_tools)

    agent("What is Bedrock AgentCore?")

## What is Bedrock AgentCore Identity?

Amazon Bedrock AgentCore Identity provides secure credential management for AI agents that need to access external services. Key benefits include:

- **Secure Storage**: Stores API keys, tokens, and credentials in AWS Secrets Manager
- **Runtime Retrieval**: Provides secure credential access at runtime without hardcoding
- **Access Control**: Integrates with AWS IAM for fine-grained access permissions
- **Audit Trail**: Maintains logs of credential access for security monitoring
- **Rotation Support**: Enables automatic credential rotation and lifecycle management

This service eliminates the need to hardcode sensitive credentials in your application code, reducing security risks and improving compliance.

## Creating Secure Credential Management

### Step 1: Create API Key Credential Provider

Now we'll create an API Key Credential Provider using Bedrock AgentCore Identity. This securely stores the Exa API key in AWS Secrets Manager and provides a managed way to access it without exposing credentials in code.

In [ ]:
from bedrock_agentcore.services.identity import IdentityClient
import boto3

# !-------- UPDATE THE EXA API KEY HERE  --------!
EXA_API_KEY = <YOUR EXA API KEY> 

region = boto3.session.Session().region_name

#Configure API Key Provider
identity_client = IdentityClient(region=region)

try:
    api_key_provider = identity_client.create_api_key_credential_provider({
        "name": "exa-apikey-provider",
        "apiKey": EXA_API_KEY # Replace it with the API key you obtain from the external application vendor, e.g., OpenAI
    })
    print("Created AgentCore Identity API Key Credential Provider.")
    print(api_key_provider)
except Exception as e:
    print("AgentCore Identity API Key Credential Provider already exist.")

### Step 2: Test Secure Credential Retrieval

Now let's test our agent using secure credential retrieval. The `@requires_api_key` decorator automatically retrieves the API key from the credential provider at runtime, ensuring no hardcoded secrets in our code while maintaining security best practices.

In [ ]:
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient
from mcp.client.streamable_http import streamablehttp_client
from bedrock_agentcore.identity.auth import requires_api_key

@requires_api_key(provider_name="exa-apikey-provider")
def need_api_key(*, api_key: str):
    print(f'received api key for async func: {api_key}')
    return api_key

EXA_API_KEY = need_api_key()

# Connect to the EXA MCP server
print("\nConnecting to MCP Server...")
exa_server = MCPClient(lambda: streamablehttp_client(f"https://mcp.exa.ai/mcp?exaApiKey={EXA_API_KEY}"))

with exa_server:
    # Combine all tools - they all work the same way!
    mcp_tools = (
        exa_server.list_tools_sync()
    )

    print(f"Available tools: {[tool.tool_name for tool in mcp_tools]}")
    
    # Create agent with Exa MCP tools
    agent = Agent(model=BedrockModel(model_id="us.amazon.nova-pro-v1:0"),
                  system_prompt="You are a helpful assistant that provides concise responses.",
                  tools=mcp_tools)

    agent("What is Bedrock AgentCore?")

Let's examine the detailed execution flow of the agent loop to understand how the agent processes requests and generates responses:

In [ ]:
print("Agent Loop Detail")
print("-----------------")

print(f"Agent Loop Length: {len(agent.messages)})")
print("\nUser-Assistant Conversation:")
for message in agent.messages:
    print(message)

## Resource Cleanup (Optional)

Clean up the deployed resources:

In [ ]:
import boto3

agentcore_control_client = boto3.client('bedrock-agentcore-control', region_name=region)

try:
    print("Deleting AgentCore Identity...")
    agentcore_control_client.delete_api_key_credential_provider(name="exa-apikey-provider")
    print("✓ AgentCore Identity deletion initiated")
except Exception as e:
    print(f"❌ Error during cleanup: {e}")
    print("You may need to manually clean up some resources.")